# Tarea 1. Algoritmos de descenso estocástico
## Aprendizaje de Máquina I
 Esteban Reyes Saldaña

Implementar versiones estocásticas de algoritmos de descenso de gradiente

1. Descenso de gradiente estocśtico

In [1]:
def SGD (theta = [], grad = None, gd_params = [], f_params = []):
    '''
    Descenso Gradiente Estocástico
    
    PARÁMETROS
    ------------
    theta      : condición inicial
    grad       : función que calcula el gradiente
    
    gd_params  : lista de parámetros para el algoritmo de descenso,
                    nIter = gd_params['nIter'] número de iteraciones
                    alpha = gd_params['alpha'] tamaño de paso alpha
                    batch_size = gd_params['batch_size'] tamaño de la muestra
    
    f_params   : lista de parámetros para la función objetivo
                    kappa = f_params['kappa'] parámetro de escala (rechazo de outlies)
                    X     = f_params['X']     variable independiente
                    y     = f_params['y']     variable dependiente
    
    Regresa
    ------------
    Theta      : trayectoria de parametros
                    Theta[-1] es el valor alcanzado en la última iteración
    '''
    
    (high, dim) = f_params['X'].shape
    batch_size  = gd_params['batch_size']
    
    nIter       = gd_params['nIter']
    alpha       = gd_params['alpha']
    
    Theta = []
    
    for t in range(nIter):
        # Set of sampled indices
        smpIdx = np.random.randint(low = 0, high = high, size = batch_size, dtype = 'int32')
        #sample
        smpX = f_params['X'][smpIdx]
        smpy = f_params['y'][smpIdx]
        
        # parámetros de la función objetivo
        smpf_params = { **f_params, 
                       'kappa' : f_params['kappa'], 
                        'X'    : smpX , 
                        'y'    : smpy
                       }
        
        p = grad(theta, f_params = smpf_params)
        theta = theta - alpha*p
        Theta.append(theta)
        
    return np.array(Theta)

2. Descenso de gradiente estocástico accelerado de tipo Nesterov

In [2]:
def NAG(theta = [], grad = None, gd_params = {}, f_params = {}) :
    '''
    Descenso Acelerado de Nesterov
    
    Parámetros
    -----------
    theta     : condición incial
    grad      : función que calcula el gradiente
    gd_params : lista de parámetros para el algoritmo de descenso,
                    nIter = gd_params['nIter'] número de iteraciones
                    alpha = gd_params['alpha'] tamaño de paso alpha
                    eta   = gd_params['eta']   parámetro de inercia (0,1]
                    
                    batch_size = gd_params['batch_size'] tamaño de la muestra
                    
    f_params   : lista de parámetros para la función objetivo,
                    kappa = f_params['kappa'] parámetro de escala (rechazo de outliers)
                    X     = f_params['X']     variable independiente
                    y     = f_params['y']     variable dependiente
    
    Regresa
    ----------
    Theta     : trayectoria de los parámetros
                    Theta[-1] es el valor alcanzado en la última iteración
    '''
    (high, dim) = f_params['X'].shape
    batch_size  = gd_params['batch_size']
    
    nIter = gd_params['nIter']
    alpha = gd_params['alpha']
    eta   = gd_params['eta']
    
    p     = np.zeros(theta.shape)
    
    Theta = []
    
    for t in range(nIter):
        # Set of sampled indices
        smpIdx = np.random.randint(low = 0, high = high, size = batch_size, dtype = 'int32')
        #sample
        smpX = f_params['X'][smpIdx]
        smpy = f_params['y'][smpIdx]
        
        # parámetros de la función objetivo
        smpf_params = { **f_params, 
                       'kappa' : f_params['kappa'], 
                        'X'    : smpX , 
                        'y'    : smpy
                       }
        
        
        pre_theta = theta - 2.0*alpha*p
        g = grad(pre_theta, f_params = smpf_params)
        p = g + eta*p
        theta = theta - alpha*p
        Theta.append(theta)
        
    return np.array(Theta)

3. AdaDelta

In [3]:
def ADADELTA(theta = [], grad = None, gd_params = {}, f_params = {}):
    '''
    Descenso de Gradiente Adaptable (ADADELTA)
    
    Parámetros
    -----------
    theta     : Condición Inicial
    grad      : función que calcula el gradiente
    gd_params : lista de parámetros para el algoritmo de descenso,
                    nIter    = gd_params['nIter'] número de iteraciones
                    alphaADA = gd_params['alphaDELTA'] tamaño de paso alpha
                    eta      = gd_params['eta'] parámetro adaptación del alpha
    
                    batch_size = gd_params['batch_size'] tamaño de la muestra

                    
    f_params  : lista de parámetros para la función objetivo,
                    kappa = f_params['kappa'] parámetro de escala (recha<o de outliers)
                    X     = f_params['X'] variable independiente
                    y     = f_params['y'] variable dependiente
    
    Regresa
    -----------
    Theta     : trayectoria de los parámetros
                    Theta[-1] es el valor alcanzado en la última iteración
    
    '''
    (high, dim) = f_params['X'].shape
    batch_size  = gd_params['batch_size']
    
    epsilon = 1e-8
    nIter   = gd_params['nIter']
    alpha   = gd_params['alphaADADELTA']
    eta     = gd_params['eta']
    G       = np.zeros(theta.shape)
    g       = np.zeros(theta.shape)
    
    Theta = []
    
    for t in range(nIter):
        # Set of sampled indices
        smpIdx = np.random.randint(low = 0, high = high, size = batch_size, dtype = 'int32')
        #sample
        smpX = f_params['X'][smpIdx]
        smpy = f_params['y'][smpIdx]
        
        # parámetros de la función objetivo
        smpf_params = { **f_params, 
                       'kappa' : f_params['kappa'], 
                        'X'    : smpX , 
                        'y'    : smpy
                       }
        
        g = grad(theta, f_params=smpf_params)
        G = eta*g**2 + (1-eta)*G
        p = 1.0/(np.sqrt(G)+epsilon)*g
        theta = theta - alpha * p
        Theta.append(theta)
    
    return np.array(Theta)
        

4. ADAM

In [4]:
def ADAM(theta=[], grad=None, gd_params={}, f_params={}):
    '''
    Descenso de Gradiente Adaptable con Momentum (ADAM) 
    
    Parámetros
    -----------
    theta     :   condicion inicial
    grad      :   funcion que calcula el gradiente
    gd_params :   lista de parametros para el algoritmo de descenso, 
                      nIter    = gd_params['nIter'] número de iteraciones
                      alphaADA = gd_params['alphaADAM'] tamaño de paso alpha
                      eta1     = gd_params['eta1'] factor de momentum para la direccion 
                                 de descenso (0,1)
                      eta2     = gd_params['eta2'] factor de momentum para la el 
                                 tamaño de paso (0,1)
                                 
                      batch_size = gd_params['batch_size'] tamaño de la muestra
                  
    f_params  :   lista de parametros para la funcion objetivo, 
                      kappa = f_params['kappa'] parametro de escala (rechazo de outliers)
                      X     = f_params['X'] Variable independiente
                      y     = f_params['y'] Variable dependiente                   

    Regresa
    -----------
    Theta     :   trayectoria de los parametros
                     Theta[-1] es el valor alcanzado en la ultima iteracion
    '''
    (high, dim) = f_params['X'].shape
    batch_size  = gd_params['batch_size']
    
    epsilon = 1e-8
    nIter   = gd_params['nIter']
    alpha   = gd_params['alphaADAM'] 
    eta1    = gd_params['eta1']
    eta2    = gd_params['eta2']
    p       = np.zeros(theta.shape)
    v       = 0.0
    
    Theta   = []
    eta1_t  = eta1
    eta2_t  = eta2

    for t in range(nIter):
        # Set of sampled indices
        smpIdx = np.random.randint(low = 0, high = high, size = batch_size, dtype = 'int32')
        #sample
        smpX = f_params['X'][smpIdx]
        smpy = f_params['y'][smpIdx]
        
        # parámetros de la función objetivo
        smpf_params = { **f_params, 
                       'kappa' : f_params['kappa'], 
                        'X'    : smpX , 
                        'y'    : smpy
                       }

        g  = grad(theta, f_params = smpf_params)
        
        p  = eta1*p + (1.0 - eta1)*g
        v  = eta2*v + (1.0 - eta2)*(g**2)

        pp = p/(1.-eta1_t) 
        vv = v/(1.-eta2_t) 
        
        theta = theta - alpha * pp / (np.sqrt(vv) + epsilon)
        Theta.append(theta)
        
        eta1_t *= eta1
        eta2_t *= eta2

    return np.array(Theta)

5. NADAM

In [5]:
def NADAM(theta=[], grad=None, gd_params={}, f_params={}):
    '''
    Descenso de Gradiente Adaptable con Momentum (ADAM) 
    
    Parámetros
    -----------
    theta     :   condicion inicial
    grad      :   funcion que calcula el gradiente
    gd_params :   lista de parametros para el algoritmo de descenso, 
                      nIter    = gd_params['nIter'] número de iteraciones
                      alphaADA = gd_params['alphaADAM'] tamaño de paso alpha
                      eta1     = gd_params['eta1'] factor de momentum para la direccion 
                                 de descenso (0,1)
                      eta2     = gd_params['eta2'] factor de momentum para la el 
                                 tamaño de paso (0,1)
                                 
                      batch_size = gd_params['batch_size'] tamaño de la muestra
                  
    f_params  :   lista de parametros para la funcion objetivo, 
                      kappa = f_params['kappa'] parametro de escala (rechazo de outliers)
                      X     = f_params['X'] Variable independiente
                      y     = f_params['y'] Variable dependiente                   

    Regresa
    -----------
    Theta     :   trayectoria de los parametros
                     Theta[-1] es el valor alcanzado en la ultima iteracion
    '''
    (high, dim) = f_params['X'].shape
    batch_size  = gd_params['batch_size']
    
    epsilon = 1e-8
    nIter   = gd_params['nIter']
    alpha   = gd_params['alphaADAM'] 
    eta1    = gd_params['eta1']
    eta2    = gd_params['eta2']
    p       = np.zeros(theta.shape)
    v       = 0.0
    
    Theta   = []
    eta1_t  = eta1
    eta2_t  = eta2

    for t in range(nIter):
        # Set of sampled indices
        smpIdx = np.random.randint(low = 0, high = high, size = batch_size, dtype = 'int32')
        #sample
        smpX = f_params['X'][smpIdx]
        smpy = f_params['y'][smpIdx]
        
        # parámetros de la función objetivo
        smpf_params = { **f_params, 
                       'kappa' : f_params['kappa'], 
                        'X'    : smpX , 
                        'y'    : smpy
                       }
        
        
        pre_theta = theta - 2.0*alpha*p
        g  = grad(pre_theta, f_params = smpf_params)
        
        p  = eta1*p + (1.0 - eta1)*g
        v  = eta2*v + (1.0 - eta2)*(g**2)

        pp = p/(1.-eta1_t) 
        vv = v/(1.-eta2_t) 
        
        theta = theta - alpha * pp / (np.sqrt(vv) + epsilon)
        Theta.append(theta)
        
        eta1_t *= eta1
        eta2_t *= eta2
        
    return np.array(Theta)

Para resolver el problema de regresión


$$ \min_{\alpha, \mu} \dfrac{1}{2} || \Phi \alpha - y ||^2_2 $$

donde 

$$ \Phi = [\phi_1, \phi_2, \dots, \phi_n ] $$

con 

$$ \phi_{ij} = \exp \left( - \dfrac{1}{2\sigma^2} (j - \mu_i)^2 \right) $$

In [460]:
# Librerias necesarias
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable, Dict
from tabulate import tabulate

## Funciones para Actualizaciones

In [461]:
def update_phi (Y, mu, sigma, n) :
    ''' 
    Construye  Matriz de Kerneles Phi
    
    Parámetros
    -----------
        Y            : Patrones a Aproximar
        mu           : Array de medias
        sigma        : Vector de Desviaciones
        num_rad_func : Número de funciones radiales usadas
    Regresa
    -----------
        phi          : matriz de kerneles
    '''
    phi = np.zeros((Y.shape[0], n))
 
    for i in range(n):
        phi[:, i] = np.exp(- 1.0 / (2*sigma**2) * (Y - mu[i])**2)
    
    return phi

## Gradientes 

In [546]:
def grad_gaussian_radial_mu(theta, f_params) :
    '''
    Calcula el gradiente respecto a mu
    Parámetros
    -----------
        theta
        f_params : lista de parametros para la funcion objetivo, 
                      kappa = f_params['kappa'] parametro de escala (rechazo de outliers)
                      X     = f_params['X'] Variable independiente
                      y     = f_params['y'] Variable dependiente    

    Regresa
    -----------
        Array gradiente
    '''
    # Obtengo Parámetros
    phi   = f_params['X']
    alpha = f_params['Alpha']
    n     = f_params['n']
    Y     = f_params['y']
    mu    = f_params['mu']

    
    #gradient = (phi @ alpha - Y).reshape((Y.shape[0], 1)) * alpha.T * (Y.reshape((Y.shape[0], 1))* np.ones((1, n)) - np.ones((Y.shape[0], 1)) * mu.T) 
    
    gradient = (phi @ alpha - Y) @ alpha.T  * (Y @ np.ones((Y.shape[0], n)) - np.ones((1, Y.shape[0])) @ mu)

    return np.mean(gradient, axis=0)

In [547]:
def grad_gaussian_radial_alpha(theta, f_params) :
    '''
    Calcula el gradiente respecto a alpha
    Parámetros
    -----------
        theta
        f_params : lista de parametros para la funcion objetivo, 
                      kappa = f_params['kappa'] parametro de escala (rechazo de outliers)
                      X     = f_params['X'] Variable independiente
                      y     = f_params['y'] Variable dependiente    

    Regresa
    -----------
        Array gradiente
    '''
    # Obtengo Parámetros
    phi   = f_params['X']
    Y     = f_params['y']
    alpha = f_params['Alpha']
    mu    = f_params['mu']
    n     = f_params['n']
        
    # (phi (alpha) - Y) alpha^T 
    gradient = phi.T @ (phi @ alpha - Y) 
    # Y - mu^T
    return np.mean(gradient, axis = 0)

## Parámetros

In [548]:
max_iter = 100
n        = 100
m        = 2000
sigma    = 1
epsilon  = 0.01
theta    = 10 * np.random.normal(size = 2)

# parámetros del algoritmo
gd_params = {
                'alpha'         : 0.95, 
                'alphaADADELTA' : 0.7,
                'alphaADAM'     : 0.95,
                'nIter'         : 300,
                'batch_size'    : 100,
                'eta'           : 0.9,
                'eta1'          : 0.9,
                'eta2'          : 0.999
            }

# Distintos descensos de gradientes a usar
grad_algthms = [
                (SGD, 'SDG'),
                (NAG, 'NAG'), 
                (ADADELTA, 'ADADELTA'), 
                (ADAM, 'ADAM'),
                (NADAM, 'NADAM')
               ]


In [549]:
import time
import random

def solver(algthm, Y):
    t_init = time.clock_gettime(0) 
    
    # Valores Iniciales
    mu     = np.linspace(0, 100, n) 
    phi    = get_phi(Y, mu, sigma, n)
    alpha  = np.random.uniform(0, sigma, n)

    # Parámetros para el gradiente
    f_params = {
                    'kappa' : 0.01, 
                    'mu'    : mu, 
                    'X'     : phi,
                    'y'     : Y, 
                    'Alpha' : alpha, 
                    'n'     : n
                }
    
    num_iter = 0
    
    while num_iter < max_iter :
        ''' D E S C E N S O   P A R A  A L P H A '''
        alpha_prev = alpha
        alpha      = algthm(alpha, grad = grad_gaussian_radial_alpha, gd_params = gd_params, f_params = f_params)[-1]

        if np.linalg.norm(phi @ alpha - Y) < epsilon:
            break

        ''' D E S C E N S O  P A R A  M U '''
        mu_old = mu
        mu     = algthm(mu, grad = grad_gaussian_radial_mu, gd_params = gd_params, f_params = f_params)[-1]
        
        ''' A C T U A L I Z A C I Ó N '''
        phi = get_phi(Y, mu, sigma, n)
        
        # Criterio de parada
        if np.linalg.norm(mu - mu_old) < epsilon:
            break
        # Número máximo de iteraciones si no hay converrgencia
        num_iter += 1
        
            
    t_end = time.clock_gettime(0)
    
    total_time = t_end - t_init
        
    return phi, alpha, total_time

In [550]:
''' 
E J E C U C I Ó N  
'''
# Muestra aleatoria
Y       = np.random.uniform(0, 1, m)
results = [solver(algthm[0], Y) for algthm in grad_algthms]

## Resultados

In [551]:
# Compara tiempos de ejecución
times = [ 
            [algthm for (_, algthm) in grad_algthms], 
            [round(result[-1], 8) for result in results], 
            [round(np.square( result[0] @ result[1] - Y ).mean(), 8) for result in results]
        ]

times[0] = ["Algorithm"] + times[0]
times[1] = ["Time"] + times[1]
times[2] = ["Cummulated Error"] + times[2]

print(tabulate(times))

----------------  ----------  ----------  ----------  ----------  ----------
Algorithm         SDG         NAG         ADADELTA    ADAM        NADAM
Time              4.56265473  4.59919763  4.87210226  6.99568176  5.4384284
Cummulated Error  0.32567695  0.32567695  0.32567695  0.32567695  0.32567695
----------------  ----------  ----------  ----------  ----------  ----------
